# Scraping and parsing from YouTube query

In [ ]:
import requests
import json
from bs4 import BeautifulSoup

# keywords = ['python', 'youtube', 'scraper']
keywords = ['arranging flowers']
pages = 5

baseUrl = 'https://www.youtube.com'
headMatch = '{"responseContext"'
tailMatch = ';</script>'
outputFile = open('output.csv', 'w')
debugFile = open('debug.txt', 'w')
outputLines = []

for page in range(1, pages+1):
    response = requests.get(baseUrl+'/results?search_query='+'+'.join(keywords)+'&page='+str(page))
    soup = BeautifulSoup(response.text, 'html.parser')
    debugFile.write(soup.prettify())
    jsonStr = ''

    for script in soup.find_all('script'):
        scriptStr = str(script)
        headIndex = scriptStr.find(headMatch)
        if headIndex != -1:
            tailIndex = scriptStr.find(tailMatch)
            jsonStr = scriptStr[headIndex:tailIndex]
            break

    data = json.loads(jsonStr)
    contents = data['contents']['twoColumnSearchResultsRenderer']['primaryContents']['sectionListRenderer']['contents'][0]['itemSectionRenderer']['contents']
    for content in contents:
        if 'videoRenderer' in content:
            title = content['videoRenderer']['title']['runs'][0]['text'].replace(',', ' ')
            url = content['videoRenderer']['navigationEndpoint']['commandMetadata']['webCommandMetadata']['url']
            outputLines.append(title+','+baseUrl+url)

outputFile.write('title,url\n'+'\n'.join(outputLines))

9620

# Download youtube videos

Make directory for the youtube videos

In [ ]:
import os

# Directory for the YouTube videos
youtube_dir = '/content/youtube'
os.mkdir(youtube_dir)

Download YouTube videos from scraped information

In [ ]:
!pip install --upgrade youtube_dl

     |████████████████████████████████| 1.9MB 4.5MB/s 


In [ ]:
import youtube_dl, subprocess
import datetime
import numpy as np


'''
  YouTube videos
'''

links = ['https://www.youtube.com/watch?v=qZOP4ZPkIAU',
         ]
for i, link in enumerate(links):
  for j in range(0,60,2): # The range is number of seconds to test
    URL = link # url of YouTube video
    FROM = str(datetime.timedelta(seconds=j))
    LENGTH = str(datetime.timedelta(seconds=2))
    TARGET = os.path.join(youtube_dir,"test.{}.mp4".format(np.int(j/2)))
    
    with youtube_dl.YoutubeDL({'format': 'best'}) as ydl:
      try:
        result = ydl.extract_info(URL, download=False)
        video = result['entries'][0] if 'entries' in result else result
      except:
        continue
      
    url = video['url']
    subprocess.call(['ffmpeg', '-i', url, '-ss', FROM, '-t', LENGTH, 
                    '-c:v', 'copy', '-c:a', 'copy', TARGET,]) 
  

[youtube] qZOP4ZPkIAU: Downloading webpage
[youtube] qZOP4ZPkIAU: Downloading webpage
[youtube] qZOP4ZPkIAU: Downloading webpage
[youtube] qZOP4ZPkIAU: Downloading webpage
[youtube] qZOP4ZPkIAU: Downloading webpage


# Test on YouTube videos and write to .json

Load saved model

In [ ]:
import keras
from keras.models import load_model

test_model = load_model('plants_Inception_try.h5')

Function for predicting whether clip contains target action and write to file

In [ ]:
import numpy as np
import cv2
import json
from time import sleep
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt


img_rows, img_cols, img_depth = 112, 112, 25

def load_video(path, max_frames=0, resize=(112, 112)):
  cap = cv2.VideoCapture(path)
  frames = []
  try:
    while True:
      ret, frame = cap.read()
      if frame is not None:
        frame = cv2.resize(frame, resize)
        frame = frame[:, :, [2, 1, 0]]
        frames.append(frame)
        # plt.imshow(frame) 

      else:
        # print("Frame is None")
        break
      
      if len(frames) == max_frames:
        break
  finally:
    cap.release()
  return np.array(frames) / 255.0

videos = []

for i in range(30): # The number of clips downloaded to test
  try:
    frames = load_video('/content/youtube/test.{}.mp4'.format(i),max_frames=25)

    frames = np.expand_dims(frames,axis=0)

    p = test_model.predict(frames, batch_size = 1) # verbose = 1
    # print(p)
    prob = p[0][1]

    if prob > 0.6:
        temp = {
            'videoID': "qZOP4ZPkIAU", # Change to video ID
            'type': "segment",
            'startTime': np.float(i*2),
            'endTime': np.float((i+1)*2),
            'observer': 'CSCE636Spring2021-astrajoan-10',
            'isHuman': 'false',
            'confirmedBySomeone': 'false',
            'rejectedBySomeone': 'false',
            'observation': {
                'label': 'Arrange_flowers_or_water_plants_etc',
                'labelConfidence': np.float(prob),
            }
        }

        videos.append(temp)

  except:
    continue

with open('1.json', 'w', encoding='utf-8') as f:
  json.dump(videos, f, indent=4)


